In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
data = pd.read_csv('./digitRecognizer/train.csv')

In [3]:
from sklearn.preprocessing import OneHotEncoder
X, y = data.iloc[:,1:].astype('float')/255,data.iloc[:,:1]
print(X.head())
enc = OneHotEncoder()
enc.fit(y)
y = enc.transform(y).toarray()
X = np.array(X)

   pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   pixel9    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0     0.0    ...          0.0       0.0       0.0       0.0       0.0   
1     0.0    ...          0.0       0.0       0.0       0.0       0.0   
2     0.0    ...          0.0       0.0       0.0       0.0       0.0   
3     0.0    ...          0.0       0.0       0.0       0.0       0.0   
4     0.0    ...          0.0       0.0       0.0       0.0       0.0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0       0.0       0.0       0.0       0.0   

In [4]:
print(X.shape, y.shape)

(42000, 784) (42000, 10)


In [5]:
x1, x2, y1, y2 = train_test_split(X, y, test_size=0.3, random_state=3)

In [6]:
print(x1.shape, x2.shape, y1.shape, y2.shape)
print([_ for _ in map(type, [x1,x2,y1,y2])])

(29400, 784) (12600, 784) (29400, 10) (12600, 10)
[<class 'numpy.ndarray'>, <class 'numpy.ndarray'>, <class 'numpy.ndarray'>, <class 'numpy.ndarray'>]


In [7]:
sess = tf.InteractiveSession()
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))
def bias_variable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1],padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
x = tf.placeholder(tf.float32, shape=[None, 784], name='input')
y_ = tf.placeholder(tf.float32, shape=[None, 10], name='output')
x_image = tf.reshape(x, [-1, 28, 28, 1])
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool(h_conv1)
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool(h_conv2)
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)+b_fc2)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [8]:
import random
def get_random_cached(x1, y1, size):
    a, b = [],[]
    n = len(x1)
    for i in range(size):
        r = random.randint(0, n-1)
        a.append(x1[r])
        b.append(y1[r])
    return np.array(a), np.array(b)

In [10]:
tf.global_variables_initializer().run()
for i in range(2000):
    batch_x, batch_y = get_random_cached(x1, y1,30)
    if not i%100:
        print("step {0}, training accuracy is {1}".format(i, accuracy.eval(feed_dict={x:batch_x, y_:batch_y,keep_prob:1.0})))
    train_step.run(feed_dict={x:batch_x,y_:batch_y,keep_prob:0.5})
print("accuracy is {0}".format(accuracy.eval(feed_dict={x:x2,y_:y2,keep_prob:1.0})))

step 0, training accuracy is 0.03333333507180214


step 100, training accuracy is 0.8333333134651184


step 200, training accuracy is 0.9333333373069763


step 300, training accuracy is 0.8999999761581421


step 400, training accuracy is 1.0


...

step 1600, training accuracy is 0.9666666388511658


step 1700, training accuracy is 0.9666666388511658


step 1800, training accuracy is 1.0


step 1900, training accuracy is 1.0


accuracy is 0.9676983952522278


In [10]:
test = pd.read_csv('./digitRecognizer/test.csv').astype('float')
print(test.head())

   pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   pixel9    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0     0.0    ...          0.0       0.0       0.0       0.0       0.0   
1     0.0    ...          0.0       0.0       0.0       0.0       0.0   
2     0.0    ...          0.0       0.0       0.0       0.0       0.0   
3     0.0    ...          0.0       0.0       0.0       0.0       0.0   
4     0.0    ...          0.0       0.0       0.0       0.0       0.0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0       0.0       0.0       0.0       0.0   

In [11]:
test = np.array(test/255)
test[:5]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
a = sess.run(y_conv,feed_dict={x:test,keep_prob:1.0})

In [13]:
b = tf.argmax(a,1).eval()
c = pd.Series(b)
c.value_counts()

1    3210
7    2906
8    2862
9    2838
0    2832
2    2777
3    2759
6    2724
4    2686
5    2406
dtype: int64

In [14]:
c = pd.Series(b)
c.head()

0    2
1    0
2    9
3    9
4    3
dtype: int64

In [15]:
c.value_counts()

1    3210
7    2906
8    2862
9    2838
0    2832
2    2777
3    2759
6    2724
4    2686
5    2406
dtype: int64

In [16]:
a[:100]

array([[  1.41830851e-05,   8.40834957e-09,   9.99972582e-01,
          3.64916195e-06,   2.72501353e-07,   3.94751360e-08,
          7.09431561e-07,   2.26383875e-07,   6.81722668e-06,
          1.54249187e-06],
       [  9.99926805e-01,   2.40710456e-08,   1.19821907e-05,
          1.22606491e-06,   2.78070256e-09,   4.26471815e-05,
          7.34062132e-06,   5.87978229e-06,   4.33176581e-07,
          3.67899293e-06],
       [  4.92655672e-05,   2.66096613e-04,   2.99787673e-04,
          9.87147796e-04,   4.70193988e-03,   1.10536488e-03,
          1.13341121e-04,   2.23202654e-03,   3.01240496e-02,
          9.60121036e-01],
       [  7.65735954e-02,   5.94443176e-04,   1.59320652e-01,
          1.38945458e-03,   3.11199240e-02,   7.61954929e-04,
          9.13971569e-03,   8.98802802e-02,   1.08414795e-02,
          6.20378494e-01],
       [  7.54559660e-05,   3.33647593e-04,   4.70439792e-02,
          9.48975980e-01,   4.70304713e-07,   3.55923665e-04,
          5.52651836e-05

In [17]:
c.values

array([2, 0, 9, ..., 3, 9, 2])

In [20]:
df = pd.DataFrame({'imageId': [_+1 for _ in range(len(c))],'label':c.values})

In [21]:
df.to_csv('tensorflow.csv',index=False)